# Для копаний со случайным лесом и его особенностями

# Деревья и леса

Загрузите датасет digits с помощью функции load_digits из sklearn.datasets и подготовьте матрицу признаков X и ответы на обучающей выборке y (вам потребуются поля data и target в объекте, который возвращает load_digits).


In [ ]:
from sklearn import datasets
%pylab inline

In [ ]:
digits = datasets.load_digits()
digits.keys()

In [ ]:
X = digits['data']
y = digits['target']
print X
print y
print X.shape
print y.shape

In [ ]:
# Bagged Decision Trees for Classification
import pandas
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pandas.read_csv(url, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
cart = DecisionTreeClassifier()
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

__1.__ Создайте `DecisionTreeClassifier` с настройками по умолчанию и измерьте качество его работы с помощью cross_val_score. Эта величина и будет ответом в пункте 1.

In [ ]:
from sklearn import tree

tree_base_classif = tree.DecisionTreeClassifier( )
tree_base_classif.fit(X,y)

In [ ]:
from sklearn.cross_validation import cross_val_score
tree_scoring = cross_val_score(tree_base_classif, X, y, scoring = 'accuracy', cv = 10)

In [ ]:
print tree_scoring
print tree_scoring.mean()

2\. Воспользуйтесь BaggingClassifier из sklearn.ensemble, чтобы обучить бэггинг над DecisionTreeClassifier. Используйте в BaggingClassifier параметры по умолчанию, задав только количество деревьев равным 100.

Качество классификации новой модели - ответ в пункте 2. Обратите внимание, как соотносится качество работы композиции решающих деревьев с качеством работы одного решающего дерева.

In [ ]:
from sklearn import ensemble

bg_classifier = ensemble.BaggingClassifier(base_estimator = tree_base_classif, n_estimators = 100)
bg_classifier.fit(X,y)

In [ ]:
bg_scoring = cross_val_score(bg_classifier, X, y, scoring = 'accuracy', cv = 10)
print bg_scoring
print bg_scoring.mean()

3\. Теперь изучите параметры BaggingClassifier и выберите их такими, чтобы каждый базовый алгоритм обучался не на всех $d$ признаках, а на $\sqrt{d}$ случайных признаков. Качество работы получившегося классификатора - ответ в пункте 3. Корень из числа признаков - часто используемая эвристика в задачах классификации, в задачах регрессии же часто берут число признаков, деленное на три. Но в общем случае ничто не мешает вам выбирать любое другое число случайных признаков.


In [ ]:
bg_sqrtd_classifier = ensemble.BaggingClassifier(base_estimator = tree_base_classif, 
                                                 n_estimators = 100, max_features = 8)

bg_sqrtd_scoring = cross_val_score(bg_sqrtd_classifier, X, y, scoring = 'accuracy', cv = 10)
print bg_sqrtd_scoring
print bg_sqrtd_scoring.mean()

4\. Наконец, давайте попробуем выбирать случайные признаки не один раз на все дерево, а при построении каждой вершины дерева. Сделать это несложно: нужно убрать выбор случайного подмножества признаков в BaggingClassifier и добавить его в DecisionTreeClassifier. Какой параметр за это отвечает, можно понять из документации sklearn, либо просто попробовать угадать (скорее всего, у вас сразу получится). Попробуйте выбирать опять же $\sqrt{d}$ признаков. Качество полученного классификатора на контрольной выборке и будет ответом в пункте 4.

In [ ]:
tree_base_classif = tree.DecisionTreeClassifier(max_features = 'sqrt')

bg_sqrtd_classifier = ensemble.BaggingClassifier(base_estimator = tree_base_classif, 
                                                 n_estimators = 100)

bg_sqrtd_scoring = cross_val_score(bg_sqrtd_classifier, X, y, scoring = 'accuracy', cv = 10)
print bg_sqrtd_scoring
print bg_sqrtd_scoring.mean()

5\. Полученный в пункте 4 классификатор - бэггинг на рандомизированных деревьях (в которых при построении каждой вершины выбирается случайное подмножество признаков и разбиение ищется только по ним). Это в точности соответствует алгоритму Random Forest, поэтому почему бы не сравнить качество работы классификатора с RandomForestClassifier из sklearn.ensemble. 

Сделайте это, а затем изучите, как качество классификации на данном датасете зависит от количества деревьев, количества признаков, выбираемых при построении каждой вершины дерева, а также ограничений на глубину дерева. Для наглядности лучше построить графики зависимости качества от значений параметров, но для сдачи задания это делать не обязательно.

In [ ]:
rf_classifier = ensemble.RandomForestClassifier(n_estimators = 100)
rf_classifier.fit(X,y)

In [ ]:
rf_scoring = cross_val_score(rf_classifier, X, y, scoring = 'accuracy', cv = 10)
print rf_scoring
print rf_scoring.mean()

На основе наблюдений выпишите через пробел номера правильных утверждений из приведенных ниже в порядке возрастания номера (это будет ответ в п.5)

1) Случайный лес сильно переобучается с ростом количества деревьев

2) При очень маленьком числе деревьев (5, 10, 15), случайный лес работает хуже, чем при большем числе деревьев

3) С ростом количества деревьев в случайном лесе, в какой-то момент деревьев становится достаточно для высокого качества классификации, а затем качество существенно не меняется.

4) При большом количестве признаков (для данного датасета - 40, 50) качество классификации становится хуже, чем при малом количестве признаков (5, 10). Это связано с тем, что чем меньше признаков выбирается в каждом узле, тем более различными получаются деревья (ведь деревья сильно неустойчивы к изменениям в обучающей выборке), и тем лучше работает их композиция.

5) При большом количестве признаков (40, 50, 60) качество классификации лучше, чем при малом количестве признаков (5, 10). Это связано с тем, что чем больше признаков - тем больше информации об объектах, а значит алгоритм может делать прогнозы более точно.

6) При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса намного лучше, чем без ограничения глубины, т.к. деревья получаются не переобученными. С ростом глубины деревьев качество ухудшается.

7) При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса заметно хуже, чем без ограничений, т.к. деревья получаются недообученными. С ростом глубины качество сначала улучшается, а затем не меняется существенно, т.к. из-за усреднения прогнозов и различий деревьев их переобученность в бэггинге не сказывается на итоговом качестве (все деревья преобучены по-разному, и при усреднении они компенсируют переобученность друг-друга).


Правильные утверждения:  2,3,4,7

- Про кросс-валидацию по числу деревьев

Теперь подберём число деревьев (*n_estimators*) в алгоритме RandomForest. Как известно, в общем случае Random Forest не переобучается с увеличением количества деревьев. Подберите количество деревьев, начиная с которого качество на кросс-валидации стабилизируется. Обратите внимание, что для проведения этого эксперимента не нужно с нуля обучать много случайных лесов с различными количествами деревьев: обучите один случайный лес с максимальным интересным количеством деревьев, а затем рассмотрите подмножества деревьев разных размеров, состоящие из деревьев построенного леса (поле [*estimators_*](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)). В дальнейших экспериментах используйте найденное количество деревьев.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score


# создаём объект, в котором будет лежать разбиение индексов на фолды
kf = KFold(n_splits=5, random_state=s)

# Обучаем пять лесов, по одному для каждого фолда :3 
rfs = [ ]
for train,test in kf.split(y):
    clf_rf = ensemble.RandomForestClassifier(random_state=s, n_estimators=1000)
    clf_rf.fit(X_real[train],y[train])
    rfs.append(clf_rf)


def find_roc_auc(i_fold, X, y):
    """
        Находит по номеру фолда 1000 последовательных рок ауков
        На каждой итерации увеличиаваем число деревьев в лесу на одно
    """
    roc_auc = [ ] # сюда будем записывать пятёрки чисел 

    # сюда будем записывать предсказания каждого дерева
    predictions = [ ]

    # запоминаем тестовые индексы 
    test_ind = list(kf.split(y))[i_fold][-1]
    for tree in rfs[i_fold].estimators_:
        predictions.append(tree.predict(X[test_ind]))

    # делаем прогнозы нумпаевской матрицей
    predictions = np.array(predictions)

    # кумулятивная сумма прогнозов 
    Z_sum = np.cumsum(predictions,axis=0) 

    # число деревьев, которое на данной итерации принимает решение
    Z_cnt = np.ones(predictions.shape)*np.arange(1,predictions.shape[0] + 1, 1).reshape(predictions.shape[0], 1)

    # Кумулятивной среднее (Вероятность быть отнесённым к классу 1)
    Z_mean = Z_sum/Z_cnt

    # считаем roc-auc для каждого числа деревьев 
    roc_auc = [ ]
    for row in Z_mean:
        roc_auc.append(roc_auc_score(y[test_ind], row))
    return roc_auc

# Матрица из roc_auc. По строчкам фолды, по столбцам качество при ансамбле из j деревьев
ROC = np.array([find_roc_auc(i_fold, X_real, y) for i_fold in range(5)])

# находим по фолдам средний roc-auc и дисперсию
rc_mean = ROC.mean(axis=0)
rc_std = ROC.std(axis=0)

num = np.arange(1,1001,1)
plt.plot(num, rc_mean)
plt.fill_between(num, rc_mean-2*rc_std, rc_mean+2*rc_std, alpha=0.2, color='blue')
plt.show()